# MinHash-basierte Textähnlichkeitsanalyse

Dieses Notebook implementiert MinHash zur effizienten Berechnung der Jaccard-Ähnlichkeit zwischen Textdokumenten.

## Konzept
- **MinHash**: Probabilistische Datenstruktur zur Schätzung der Jaccard-Ähnlichkeit
- **Vorteil**: Skaliert sehr gut für große Dokumentsammlungen
- **Anwendung**: Direkte Textanalyse der OPAL-Materialien
- **Output**: Ähnlichkeitsmatrix basierend auf Textinhalt

In [1]:
# Import aller benötigten Bibliotheken für MinHash
import pandas as pd
import numpy as np
from pathlib import Path
import re
import gc
from datasketch import MinHash
from tqdm import tqdm

In [2]:

import sys
sys.path.append('../../src')
from DataHandler import DataHandler

dataHandler = DataHandler("../config.yaml")
config_manager = dataHandler.config_manager

In [3]:
# Lade OPAL-Metadaten und konfiguriere Content-Ordner
df_aimeta = dataHandler.load_data("data_files.raw_data.df_aimeta")
content_folder = dataHandler.config_manager.get('folder_structure.raw_content')

print(f"📊 Geladen: {len(df_aimeta)} OPAL-Materialien")
print(f"📁 Content-Ordner: {content_folder}")

📂 Lade: OPAL_ai_meta.p
   ✅ Geladen: 4,548 Zeilen × 25 Spalten
📊 Geladen: 4548 OPAL-Materialien
📁 Content-Ordner: /media/sz/Data/Connected_Lecturers/Opal/raw/content


In [4]:
def clean_text(text):
    """Bereinigt Text für MinHash-Verarbeitung"""
    if pd.isna(text) or text is None:
        return ""
    
    # Konvertiere zu String falls noch nicht
    text = str(text)
    
    # Kleinbuchstaben
    text = text.lower()
    
    # Entferne Interpunktion und Sonderzeichen
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Entferne mehrfache Leerzeichen
    text = re.sub(r'\s+', ' ', text)
    
    # Entferne führende/nachfolgende Leerzeichen
    text = text.strip()
    
    return text

def create_shingles(text, k=3):
    """Erstellt k-Shingles aus Text"""
    if not text or len(text) < k:
        return set()
    
    words = text.split()
    if len(words) < k:
        return {' '.join(words)}
    
    shingles = set()
    for i in range(len(words) - k + 1):
        shingle = ' '.join(words[i:i+k])
        shingles.add(shingle)
    
    return shingles

def load_content_file(pipe_id, content_folder):
    """Lädt den Inhalt einer Content-Datei basierend auf pipe:ID"""
    content_path = Path(content_folder)
    
    # Suche nach Datei mit pipe_id im Namen
    # Format ist meist: {pipe_id}.txt oder ähnlich
    possible_files = list(content_path.glob(f"*{pipe_id}*"))
    
    if not possible_files:
        return ""
    
    # Nimm die erste gefundene Datei
    file_path = possible_files[0]
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except Exception as e:
        print(f"   ⚠️ Fehler beim Laden von {file_path}: {e}")
        return ""

print("\n📋 SCHRITT 1: DATEN VORBEREITEN")
print("-" * 40)

# Verwende alle verfügbaren Materialien
valid_materials = df_aimeta.copy()
print(f"🔢 Anzahl zu verarbeitender Materialien: {len(valid_materials)}")

# Erstelle Mapping von Index zu pipe:ID
pipe_ids = valid_materials['pipe:ID'].tolist()
print(f"📋 pipe:ID Bereich: {min(pipe_ids)} bis {max(pipe_ids)}")

print("\n⚙️ KONFIGURATION")
print("-" * 40)

# Konfiguration für Text-Features (anpassbar)
INCLUDE_CONTENT = True      # Haupttext-Content einbeziehen
INCLUDE_TITLE = False        # Titel einbeziehen
INCLUDE_KEYWORDS = False     # Keywords einbeziehen

# Gewichtung der verschiedenen Textquellen
TITLE_WEIGHT = 3           # Titel 3x gewichten (da sehr relevant)
KEYWORDS_WEIGHT = 2        # Keywords 2x gewichten (wichtige Schlagwörter)

# Konfiguration für Shingles
k_shingles = 3  # 3-gram Shingles
num_perm = 128  # Anzahl Permutationen für MinHash

print(f"📄 Text-Features Konfiguration:")
print(f"   📖 Content einbeziehen: {'✅' if INCLUDE_CONTENT else '❌'}")
print(f"   📝 Titel einbeziehen: {'✅' if INCLUDE_TITLE else '❌'} (Gewichtung: {TITLE_WEIGHT}x)")
print(f"   🏷️  Keywords einbeziehen: {'✅' if INCLUDE_KEYWORDS else '❌'} (Gewichtung: {KEYWORDS_WEIGHT}x)")
print(f"   ⚙️ Shingle-Konfiguration: {k_shingles}-gram Shingles, {num_perm} Permutationen")

print("\n🔤 SCHRITT 2: TEXT-FEATURES EXTRAHIEREN")
print("-" * 40)

# Sammle alle Text-Features
all_documents = {}
failed_loads = 0

print("\n📄 Lade Content-Dateien und kombiniere mit Metadaten...")

for idx, (df_idx, row) in enumerate(tqdm(valid_materials.iterrows(), total=len(valid_materials), desc="Verarbeite Materialien")):
    pipe_id = row['pipe:ID']
    
    # 1. Lade Content-Datei (optional)
    content_text = ""
    if INCLUDE_CONTENT:
        content_text = load_content_file(pipe_id, content_folder)
        if not content_text.strip():
            failed_loads += 1
    
    # 2. Extrahiere Titel (optional)
    title = ""
    if INCLUDE_TITLE:
        title = row.get('ai:title', '')
        if pd.isna(title):
            title = ''
    
    # 3. Extrahiere Keywords (optional) - verbesserte Behandlung für Arrays
    keywords_text = ""
    if INCLUDE_KEYWORDS:
        keywords = row.get('valid_ddc_keywords', [])
        
        try:
            if keywords is not None and hasattr(keywords, '__iter__') and not isinstance(keywords, str):
                # Es ist eine Liste/Array
                keywords_list = [str(kw).strip() for kw in keywords if kw is not None and str(kw).strip()]
                keywords_text = ' '.join(keywords_list)
            elif keywords is not None:
                # Es ist ein einzelner Wert
                keywords_text = str(keywords).strip()
        except Exception as e:
            # Falls etwas schiefgeht, ignoriere Keywords für dieses Dokument
            keywords_text = ""
    
    # 4. Kombiniere alle aktivierten Textquellen
    combined_text_parts = []
    
    if INCLUDE_CONTENT and content_text.strip():
        combined_text_parts.append(content_text)
    
    if INCLUDE_TITLE and title.strip():
        # Titel entsprechend der Gewichtung hinzufügen
        combined_text_parts.extend([title] * TITLE_WEIGHT)
    
    if INCLUDE_KEYWORDS and keywords_text.strip():
        # Keywords entsprechend der Gewichtung hinzufügen
        combined_text_parts.extend([keywords_text] * KEYWORDS_WEIGHT)
    
    # Kombiniere zu einem Dokument
    if combined_text_parts:
        combined_text = ' '.join(combined_text_parts)
        cleaned_text = clean_text(combined_text)
        
        # Erstelle Shingles
        shingles = create_shingles(cleaned_text, k_shingles)
        
        if shingles:  # Nur wenn Shingles vorhanden
            all_documents[pipe_id] = shingles
    
    # Memory cleanup alle 500 Iterationen
    if idx % 500 == 0:
        gc.collect()

print(f"\n✅ Erfolgreich verarbeitete Materialien: {len(all_documents)}")
if INCLUDE_CONTENT:
    print(f"❌ Fehlgeschlagene Content-Loads: {failed_loads}")
    print(f"📊 Content-Verfügbarkeitsrate: {(len(valid_materials)-failed_loads)/len(valid_materials)*100:.1f}%")
print(f"📊 Gesamt-Verarbeitungsrate: {len(all_documents)/len(valid_materials)*100:.1f}%")

# Zeige verwendete Text-Features
print(f"\n📝 Verwendete Text-Features:")
active_features = []
if INCLUDE_CONTENT:
    active_features.append(f"Content (Haupttext)")
if INCLUDE_TITLE:
    active_features.append(f"Titel (Gewichtung: {TITLE_WEIGHT}x)")
if INCLUDE_KEYWORDS:
    active_features.append(f"Keywords (Gewichtung: {KEYWORDS_WEIGHT}x)")

for i, feature in enumerate(active_features, 1):
    print(f"   {i}. {feature}")

if not active_features:
    print("   ⚠️ WARNUNG: Keine Text-Features aktiviert!")

# Zeige Statistiken
if all_documents:
    shingle_counts = [len(shingles) for shingles in all_documents.values()]
    print(f"📈 Shingle-Statistiken:")
    print(f"   Durchschnitt: {np.mean(shingle_counts):.1f}")
    print(f"   Median: {np.median(shingle_counts):.1f}")
    print(f"   Min: {min(shingle_counts)}")
    print(f"   Max: {max(shingle_counts)}")

print(f"\n💾 Verwende {len(all_documents)} Materialien für MinHash-Berechnung")


📋 SCHRITT 1: DATEN VORBEREITEN
----------------------------------------
🔢 Anzahl zu verarbeitender Materialien: 4548
📋 pipe:ID Bereich: 1-FnRgnGtuu4 bis 9zrgYh873fJ4

⚙️ KONFIGURATION
----------------------------------------
📄 Text-Features Konfiguration:
   📖 Content einbeziehen: ✅
   📝 Titel einbeziehen: ❌ (Gewichtung: 3x)
   🏷️  Keywords einbeziehen: ❌ (Gewichtung: 2x)
   ⚙️ Shingle-Konfiguration: 3-gram Shingles, 128 Permutationen

🔤 SCHRITT 2: TEXT-FEATURES EXTRAHIEREN
----------------------------------------

📄 Lade Content-Dateien und kombiniere mit Metadaten...


Verarbeite Materialien:   0%|          | 0/4548 [00:00<?, ?it/s]

Verarbeite Materialien: 100%|██████████| 4548/4548 [00:42<00:00, 106.06it/s]


✅ Erfolgreich verarbeitete Materialien: 4548
❌ Fehlgeschlagene Content-Loads: 0
📊 Content-Verfügbarkeitsrate: 100.0%
📊 Gesamt-Verarbeitungsrate: 100.0%

📝 Verwendete Text-Features:
   1. Content (Haupttext)
📈 Shingle-Statistiken:
   Durchschnitt: 1771.2
   Median: 714.5
   Min: 22
   Max: 83437

💾 Verwende 4548 Materialien für MinHash-Berechnung


In [5]:
print("\n🧮 SCHRITT 3: MINHASH-BERECHNUNG")
print("-" * 40)

# Erstelle MinHash-Signaturen für alle Dokumente
document_minhashes = {}
document_ids = list(all_documents.keys())

print(f"🔢 Erstelle MinHash-Signaturen für {len(document_ids)} Dokumente...")

for idx, doc_id in enumerate(tqdm(document_ids, desc="MinHash Signaturen")):
    # Erstelle MinHash für dieses Dokument
    m = MinHash(num_perm=num_perm)
    
    # Füge alle Shingles zu MinHash hinzu
    for shingle in all_documents[doc_id]:
        m.update(shingle.encode('utf8'))
    
    document_minhashes[doc_id] = m
    
    # Memory cleanup alle 1000 Dokumente
    if idx % 1000 == 0:
        gc.collect()

print(f"✅ MinHash-Signaturen erstellt für {len(document_minhashes)} Dokumente")

print("\n📐 SCHRITT 4: ÄHNLICHKEITSMATRIX BERECHNEN")
print("-" * 40)

# Erstelle Ähnlichkeitsmatrix
print(f"🔢 Berechne {len(document_ids)} × {len(document_ids)} Ähnlichkeitsmatrix...")
print(f"💭 Erwartete Berechnungen: {len(document_ids) * (len(document_ids) - 1) // 2:,} Paare")

# Initialisiere Matrix
similarity_matrix = np.zeros((len(document_ids), len(document_ids)))

# Berechne Ähnlichkeiten
total_comparisons = len(document_ids) * (len(document_ids) - 1) // 2
processed_comparisons = 0

print("🔄 Berechne paarweise Ähnlichkeiten...")

with tqdm(total=total_comparisons, desc="Ähnlichkeiten") as pbar:
    for i in range(len(document_ids)):
        doc_id_1 = document_ids[i]
        minhash_1 = document_minhashes[doc_id_1]
        
        # Setze Diagonale auf 1.0 (Dokument ist mit sich selbst identisch)
        similarity_matrix[i, i] = 1.0
        
        for j in range(i + 1, len(document_ids)):
            doc_id_2 = document_ids[j]
            minhash_2 = document_minhashes[doc_id_2]
            
            # Berechne Jaccard-Ähnlichkeit
            jaccard_sim = minhash_1.jaccard(minhash_2)
            
            # Symmetrische Matrix
            similarity_matrix[i, j] = jaccard_sim
            similarity_matrix[j, i] = jaccard_sim
            
            processed_comparisons += 1
            pbar.update(1)
        
        # Memory cleanup alle 100 Zeilen
        if i % 100 == 0:
            gc.collect()

print(f"✅ Ähnlichkeitsmatrix berechnet: {similarity_matrix.shape}")

# Statistiken der Ähnlichkeitsmatrix
non_diagonal_values = similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]
print(f"\n📊 ÄHNLICHKEITSMATRIX-STATISTIKEN:")
print(f"   📏 Matrix-Größe: {similarity_matrix.shape[0]} × {similarity_matrix.shape[1]}")
print(f"   🔢 Gesamte Einträge: {similarity_matrix.size:,}")
print(f"   📈 Ähnlichkeits-Statistiken (ohne Diagonale):")
print(f"      Durchschnitt: {np.mean(non_diagonal_values):.4f}")
print(f"      Median: {np.median(non_diagonal_values):.4f}")
print(f"      Std-Abweichung: {np.std(non_diagonal_values):.4f}")
print(f"      Min: {np.min(non_diagonal_values):.4f}")
print(f"      Max: {np.max(non_diagonal_values):.4f}")

print(f"\n💾 Bereite finale DataFrames vor...")


🧮 SCHRITT 3: MINHASH-BERECHNUNG
----------------------------------------
🔢 Erstelle MinHash-Signaturen für 4548 Dokumente...


MinHash Signaturen: 100%|██████████| 4548/4548 [01:12<00:00, 62.96it/s] 


✅ MinHash-Signaturen erstellt für 4548 Dokumente

📐 SCHRITT 4: ÄHNLICHKEITSMATRIX BERECHNEN
----------------------------------------
🔢 Berechne 4548 × 4548 Ähnlichkeitsmatrix...
💭 Erwartete Berechnungen: 10,339,878 Paare
🔄 Berechne paarweise Ähnlichkeiten...


Ähnlichkeiten: 100%|██████████| 10339878/10339878 [00:50<00:00, 206443.45it/s]


✅ Ähnlichkeitsmatrix berechnet: (4548, 4548)

📊 ÄHNLICHKEITSMATRIX-STATISTIKEN:
   📏 Matrix-Größe: 4548 × 4548
   🔢 Gesamte Einträge: 20,684,304
   📈 Ähnlichkeits-Statistiken (ohne Diagonale):
      Durchschnitt: 0.0005
      Median: 0.0000
      Std-Abweichung: 0.0097
      Min: 0.0000
      Max: 1.0000

💾 Bereite finale DataFrames vor...


In [6]:
print("\n💾 SCHRITT 5: FINALE DATAFRAMES ERSTELLEN UND SPEICHERN")
print("-" * 60)

# Erstelle DataFrame für Ähnlichkeitsmatrix
print("📊 Erstelle Ähnlichkeits-DataFrame mit pipe:IDs...")
df_similarity = pd.DataFrame(
    similarity_matrix,
    index=document_ids,
    columns=document_ids
)

print(f"✅ DataFrame erstellt:")
print(f"   📊 Ähnlichkeitsmatrix: {df_similarity.shape}")

# Speichere die Matrix
print(f"\n💾 Speichere Matrix...")
dataHandler.save_data(df_similarity, "data_files.processed_data.similarity_content_based.df_minhash_text_similarity")

print(f"✅ Ähnlichkeitsmatrix erfolgreich gespeichert!")

# Cleanup großer Variablen
del document_minhashes, all_documents, similarity_matrix
gc.collect()

print(f"\n🧹 Memory cleanup durchgeführt")
print(f"\n🎉 MinHash-Analyse abgeschlossen!")


💾 SCHRITT 5: FINALE DATAFRAMES ERSTELLEN UND SPEICHERN
------------------------------------------------------------
📊 Erstelle Ähnlichkeits-DataFrame mit pipe:IDs...
✅ DataFrame erstellt:
   📊 Ähnlichkeitsmatrix: (4548, 4548)

💾 Speichere Matrix...
💾 Datei gespeichert: minhash_text_similarity.p
   📁 Pfad: /media/sz/Data/Connected_Lecturers/Opal/processed/similarity/minhash_text_similarity.p
   📊 DataFrame: 4,548 Zeilen × 4548 Spalten
   📏 Dateigröße: 157.9 MB
   🕐 Zeitstempel: 2025-07-29 15:04:08
   ⏱️  Speicherdauer: 0.06 Sekunden
✅ Ähnlichkeitsmatrix erfolgreich gespeichert!

🧹 Memory cleanup durchgeführt

🎉 MinHash-Analyse abgeschlossen!
